In [232]:
# 01_settings_check | Proje Başlangıç Ayarları ve Veri Yükleme

import os, json, sys, subprocess
from datetime import datetime
import pandas as pd
import openai
from openai import OpenAI

# Gerekli kütüphaneleri kontrol et ve kur
for pkg in ("pandas", "openai"):
    try:
        __import__(pkg)
    except ImportError:
        print(f"'{pkg}' paketi yüklendi.")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

print("Pandas Sürümü:", pd.__version__, "| OpenAI Sürümü:", openai.__version__)

# OpenAI Client başlatma ve API anahtarı kontrolü
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise EnvironmentError("OPENAI_API_KEY ortam değişkeni bulunamadı. Lütfen ayarlayın.")

client = OpenAI(api_key=api_key)
print("OpenAI istemcisi başlatıldı.")

# Hedef model
TARGET_MODEL = "gpt-4o-mini-2024-07-18"
print(f"Hedef Model: {TARGET_MODEL}")

# Veri dosyası kontrolü ve yükleme
DATA_PATH = "cleaned_result.json"
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"'{DATA_PATH}' dosyası bulunamadı. Dizini kontrol edin.")

try:
    with open(DATA_PATH, encoding="utf-8") as f:
        raw_data = json.load(f)
    df = pd.DataFrame(raw_data)
    print(f"'{DATA_PATH}' dosyasından {len(df)} kayıt yüklendi.")
except json.JSONDecodeError:
    raise ValueError(f"'{DATA_PATH}' geçerli JSON formatında değil.")
except Exception as e:
    raise Exception(f"Veri yükleme hatası: {e}")

# PDF'te belirtilen zorunlu sütunların kontrolü
required_columns = [
    "İş ID", "Lokasyon", "Sektör", "İş Gönderim Saati", "İade Talebi Saati",
    "Hizmet Veren Geri Arama Saatleri", "Hizmet Veren Arama Metinleri",
    "Partner Geri Arama Saatleri", "Partner Arama Metinleri",
    "İade Talebi Nedeni", "Partner Red Sebebi", "Müşteri Cevabı", "Sistem Cevabı",
    "İade Durumu"
]

missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    print(f"UYARI: Eksik sütunlar: {', '.join(missing_cols)}")
else:
    print("Tüm gerekli sütunlar mevcut.")

# 'İade Durumu' filtresi (PDF gereksinimi)
valid_statuses = ["Personel Kabul Etti", "Personel Redetti"]
df_filtered = df[df["İade Durumu"].isin(valid_statuses)].copy()

print(f"\nVeri Yükleme ve Filtreleme Özeti:")
print(f"  Toplam kayıt: {len(df)}")
print(f"  Filtrelenmiş kayıt: {len(df_filtered)}")

# 'İade Durumu' dağılımı ve dengesizlik kontrolü
status_counts = df_filtered['İade Durumu'].value_counts()
print(f"  'İade Durumu' dağılımı:\n{status_counts}")

min_count = status_counts.min()
max_count = status_counts.max()
if min_count / max_count < 0.2:
    print(f"\nUYARI: 'İade Durumu' sınıfları arasında önemli dengesizlik var. Performansı etkileyebilir.")

# İşlem yapılacak DataFrame'i güncelle
df = df_filtered.copy()
del df_filtered

print(f"\nİşlenecek DataFrame boyutu: {len(df)} kayıt")
df.head(3)

Pandas Sürümü: 2.3.1 | OpenAI Sürümü: 1.97.1
OpenAI istemcisi başlatıldı.
Hedef Model: gpt-4o-mini-2024-07-18
'cleaned_result.json' dosyasından 3000 kayıt yüklendi.
Tüm gerekli sütunlar mevcut.

Veri Yükleme ve Filtreleme Özeti:
  Toplam kayıt: 3000
  Filtrelenmiş kayıt: 2531
  'İade Durumu' dağılımı:
İade Durumu
Personel Kabul Etti    2303
Personel Redetti        228
Name: count, dtype: int64

UYARI: 'İade Durumu' sınıfları arasında önemli dengesizlik var. Performansı etkileyebilir.

İşlenecek DataFrame boyutu: 2531 kayıt


,İş ID,Lokasyon,Sektör,İş Gönderim Saati,İade Talebi Saati,Hizmet Veren Geri Arama Saatleri,Hizmet Veren Arama Metinleri,Partner Geri Arama Saatleri,Partner Arama Metinleri,İade Durumu,İade Talebi Nedeni,Partner Red Sebebi,Müşteri Cevabı,Sistem Cevabı
0,115607,Şanlıurfa Haliliye,Buzdolabı Servisi,2025-07-22 16:00:02,2025-07-22 16:32:59,2025-07-22 16:30:46,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,None,Personel Kabul Etti,Servisnucreti kabul. Etmiyor iade istiyorum,None,None,None
1,115605,İstanbul Pendik,Bulaşık Makinesi Servisi,2025-07-22 15:58:56,2025-07-22 17:27:46,2025-07-22 17:26:54,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,None,Personel Kabul Etti,Müşteri dün kendisi halletmiş su kapalıymış su...,None,None,İş gönderimi ile müşterinin aranma saati arası...
2,115603,Balıkesir Karesi,Klima Montajı,2025-07-22 15:28:35,2025-07-22 15:38:12,2025-07-22 15:33:56,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,None,Personel Kabul Etti,Müşteri Vazgeçti - Müşteri iptal etti Bilgi...,None,None,None


In [233]:
# 02_data_cleaning | Veri Temizleme ve Temel Özellik Mühendisliği

# Tarih sütunlarını datetime'a çevir
date_cols = ["İş Gönderim Saati", "İade Talebi Saati"]
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce")

# Metin sütunlarındaki NaN'ları boş string yap
text_cols = [
    "İade Talebi Nedeni", "Hizmet Veren Arama Metinleri", "Partner Arama Metinleri",
    "Partner Red Sebebi", "Müşteri Cevabı", "Sistem Cevabı", "Lokasyon", "Sektör"
]
for c in text_cols:
    df[c] = df[c].fillna("").astype(str)

# Geçen süreyi (dakika) hesapla
df["Time_Elapsed_Minutes"] = (
    (df["İade Talebi Saati"] - df["İş Gönderim Saati"]).dt.total_seconds() / 60
)

# Geçen süredeki NaN ve negatif değerleri düzenle
df["Time_Elapsed_Minutes"] = df["Time_Elapsed_Minutes"].fillna(-1) # NaN'ları -1 ile doldur
df["Time_Elapsed_Minutes"] = df["Time_Elapsed_Minutes"].apply(lambda x: max(0, x)) # Negatifleri 0 yap

# Tekrarlayan 'İade Durumu' filtrelemesi önceki hücrede yapıldığı için buradan kaldırıldı.

print("Veri temizliği ve temel özellik mühendisliği tamamlandı.")
print(f"Güncel kayıt sayısı: {len(df)}")
print(f"'Time_Elapsed_Minutes' sütununda NaN kontrolü: {df['Time_Elapsed_Minutes'].isnull().sum()}") # NaN kontrolü
print(f"'Time_Elapsed_Minutes' sütununun ilk 5 değeri:\n{df['Time_Elapsed_Minutes'].head()}")

df.head(3)

Veri temizliği ve temel özellik mühendisliği tamamlandı.
Güncel kayıt sayısı: 2531
'Time_Elapsed_Minutes' sütununda NaN kontrolü: 0
'Time_Elapsed_Minutes' sütununun ilk 5 değeri:
0    32.950000
1    88.833333
2     9.616667
3     5.850000
4     7.200000
Name: Time_Elapsed_Minutes, dtype: float64


,İş ID,Lokasyon,Sektör,İş Gönderim Saati,İade Talebi Saati,Hizmet Veren Geri Arama Saatleri,Hizmet Veren Arama Metinleri,Partner Geri Arama Saatleri,Partner Arama Metinleri,İade Durumu,İade Talebi Nedeni,Partner Red Sebebi,Müşteri Cevabı,Sistem Cevabı,Time_Elapsed_Minutes
0,115607,Şanlıurfa Haliliye,Buzdolabı Servisi,2025-07-22 16:00:02,2025-07-22 16:32:59,2025-07-22 16:30:46,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Servisnucreti kabul. Etmiyor iade istiyorum,,,,32.950000
1,115605,İstanbul Pendik,Bulaşık Makinesi Servisi,2025-07-22 15:58:56,2025-07-22 17:27:46,2025-07-22 17:26:54,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Müşteri dün kendisi halletmiş su kapalıymış su...,,,İş gönderimi ile müşterinin aranma saati arası...,88.833333
2,115603,Balıkesir Karesi,Klima Montajı,2025-07-22 15:28:35,2025-07-22 15:38:12,2025-07-22 15:33:56,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Müşteri Vazgeçti - Müşteri iptal etti Bilgi...,,,,9.616667


In [234]:
# 03_feature_call_counts | Arama sayılarını hesapla

def count_calls(value):
    """
    Hizmet veren veya partner tarafından yapılan arama sayısını hesaplar.
    Boş veya NaN ise 0 döner. Aksi takdirde '||' ayracına göre sayar.
    """
    if pd.isna(value) or value == "":
        return 0
    return len(str(value).split("||")) # Güvenli dönüşüm

# Yeni sayısal sütunları oluştur
df["HizmetVerenAramaSayisi"] = df["Hizmet Veren Arama Metinleri"].apply(count_calls)
df["PartnerAramaSayisi"] = df["Partner Arama Metinleri"].apply(count_calls)

print("Arama sayıları özellikleri eklendi.")
print("Arama Sayıları İstatistikleri:")
print(df[["HizmetVerenAramaSayisi", "PartnerAramaSayisi"]].describe().to_string()) # Tam tabloyu göstermek için to_string()

df.head(3)


Arama sayıları özellikleri eklendi.
Arama Sayıları İstatistikleri:
       HizmetVerenAramaSayisi  PartnerAramaSayisi
count             2531.000000         2531.000000
mean                 1.882655            0.835243
std                  1.547936            1.298874
min                  0.000000            0.000000
25%                  1.000000            0.000000
50%                  1.000000            0.000000
75%                  2.000000            1.000000
max                 24.000000           17.000000


,İş ID,Lokasyon,Sektör,İş Gönderim Saati,İade Talebi Saati,Hizmet Veren Geri Arama Saatleri,Hizmet Veren Arama Metinleri,Partner Geri Arama Saatleri,Partner Arama Metinleri,İade Durumu,İade Talebi Nedeni,Partner Red Sebebi,Müşteri Cevabı,Sistem Cevabı,Time_Elapsed_Minutes,HizmetVerenAramaSayisi,PartnerAramaSayisi
0,115607,Şanlıurfa Haliliye,Buzdolabı Servisi,2025-07-22 16:00:02,2025-07-22 16:32:59,2025-07-22 16:30:46,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Servisnucreti kabul. Etmiyor iade istiyorum,,,,32.950000,1,0
1,115605,İstanbul Pendik,Bulaşık Makinesi Servisi,2025-07-22 15:58:56,2025-07-22 17:27:46,2025-07-22 17:26:54,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Müşteri dün kendisi halletmiş su kapalıymış su...,,,İş gönderimi ile müşterinin aranma saati arası...,88.833333,1,0
2,115603,Balıkesir Karesi,Klima Montajı,2025-07-22 15:28:35,2025-07-22 15:38:12,2025-07-22 15:33:56,Görüşmeleriniz kayıt altına alınmaktadır. Lütf...,None,,Personel Kabul Etti,Müşteri Vazgeçti - Müşteri iptal etti Bilgi...,,,,9.616667,1,0


In [235]:
# 04_filter_and_prompt | system/user/assistant mesajı + JSONL oluştur

import json, pathlib

# Boş veya NaN değerler için placeholder döndürür
def default(val, placeholder="(no data)"):
    if pd.isna(val) or str(val).strip() == "":
        return placeholder
    return str(val).strip()

# SYSTEM PROMPT: AI asistanının rolünü ve beklentilerini tanımlar
SYSTEM_PROMPT = (
    "You are an AI assistant representing the internal personnel of a service platform. "
    "Your role is to review refund requests submitted by service providers and make the final decision: "
    "'Accepted by Personnel' or 'Rejected by Personnel'. "
    "You must base your decision on the following structured information:\n\n"
    "- Reason for Refund Request: A short written justification from the provider.\n"
    "- Service Provider Call Transcripts: Transcript(s) of provider's calls with the customer.\n"
    "- Service Provider Call Count: How many times the provider attempted to reach the customer (based on call segments).\n"
    "- Partner Call Transcripts: Transcript(s) of the partner's confirmation calls with the customer.\n"
    "- Partner Call Count: How many times the partner called the customer for confirmation (based on call segments).\n"
    "- Time Elapsed Between Job Submission and Refund Request: Time in minutes.\n"
    "- Partner Rejection Reason: The reason provided by the partner if they rejected the request.\n"
    "- Customer Response: Partner's notes from customer interaction.\n"
    "- System Response: Notes from automated rules.\n\n"
    "Analyze whether the provider gives a valid, timely, and reasonable explanation. "
    "Reject vague, insufficient, abusive claims, or cases where fault lies with the provider. "
    "Accept cases with clear external causes (e.g., customer unavailable, location inaccessible, technical issues beyond provider control). "
    "Do not assume anything beyond the given data.\n\n"
    "Return explanation in Turkish. Cevap açıklaması kısa ve Türkçe olmalıdır.\n\n"
    "Respond ONLY with a JSON object:\n"
    "{'decision': 'Accepted by Personnel' or 'Rejected by Personnel', 'reason': 'short explanation in Turkish'}"
)

out = []

for _, row in df.iterrows():
    # USER MESAJI: Modele sağlanacak input bilgilerini oluştur
    user_parts = [
        f"Reason for Refund Request: {default(row['İade Talebi Nedeni'])}",
        f"Service Provider Call Transcripts: {default(row['Hizmet Veren Arama Metinleri'])}",
        f"Service Provider Call Count: {default(row['HizmetVerenAramaSayisi'])}",
        f"Partner Call Transcripts: {default(row['Partner Arama Metinleri'])}",
        f"Partner Call Count: {default(row['PartnerAramaSayisi'])}",
        f"Time Elapsed Between Job Submission and Refund Request: {int(row['Time_Elapsed_Minutes'])} minutes.",
        f"Partner Rejection Reason: {default(row['Partner Red Sebebi'])}",
        f"Customer Response: {default(row['Müşteri Cevabı'])}",
        f"System Response: {default(row['Sistem Cevabı'])}"
    ]
    user_msg = "\n".join(user_parts) # Tüm user_parts'ları birleştir

    # ASSISTANT KARARI: Eğitim verisindeki doğru karar
    decision = (
        "Accepted by Personnel"
        if row["İade Durumu"] == "Personel Kabul Etti"
        else "Rejected by Personnel"
    )

    # ASSISTANT AÇIKLAMASI (REASON): Kararın Türkçe ve açıklayıcı gerekçesi
    reason_parts = []
    primary_reason_text = default(row['İade Talebi Nedeni'], "").lower()

    if "müşteri vazgeçti" in primary_reason_text:
        reason_parts.append("Müşteri vazgeçtiği belirtildi.")
    elif "ulaşılamıyor" in primary_reason_text:
        reason_parts.append("Müşteriye ulaşılamadığı bildirildi.")
    elif "fiyat" in primary_reason_text or "ücret" in primary_reason_text:
        reason_parts.append("Müşteri fiyatı kabul etmedi.")
    elif "parça yok" in primary_reason_text or "yedek parça" in primary_reason_text:
        reason_parts.append("Yedek parça bulunamadı.")
    else:
        reason_parts.append("İade talebi incelendi.")

    partner_red_text = default(row['Partner Red Sebebi'], "")
    if partner_red_text != "(no data)":
        reason_parts.append(f"Partner red gerekçesi: {partner_red_text}.")
    
    customer_response_text = default(row['Müşteri Cevabı'], "")
    if customer_response_text != "(no data)":
        reason_parts.append(f"Müşteri cevabı: {customer_response_text}.")
    
    system_response_text = default(row['Sistem Cevabı'], "")
    if system_response_text != "(no data)":
        reason_parts.append(f"Sistem cevabı: {system_response_text}.")

    reason_str = " ".join(reason_parts).strip()
    if not reason_str:
        reason_str = "Tüm bilgiler ışığında karar verildi."
    
    if len(reason_str) > 100:
        reason_str = reason_str[:97] + "..."

    # ASSISTANT MESAJINI JSON formatında oluştur
    assistant_msg = json.dumps(
        {"decision": decision, "reason": reason_str},
        ensure_ascii=False,
        separators=(",", ": ")
    )

    # FINE-TUNING İÇİN MESAJLARI OLUŞTUR ve listeye ekle
    out.append(json.dumps(
        {"messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": assistant_msg}
        ]},
        ensure_ascii=False
    ))

# JSONL dosyasını kaydet
JSONL_PATH = "fine_tuning_data.jsonl"
pathlib.Path(JSONL_PATH).write_text("\n".join(out), encoding="utf-8")

print(f"JSONL dosyası yazıldı: {JSONL_PATH}")
print(f"Oluşturulan toplam satır sayısı: {len(out)}")

JSONL dosyası yazıldı: fine_tuning_data.jsonl
Oluşturulan toplam satır sayısı: 2531


In [236]:
# 05_fine_tune_start | JSONL dosyasını yükle ve fine-tuning başlat

import openai, os

# JSONL dosyası yoksa hata ver
if not os.path.exists(JSONL_PATH):
    raise FileNotFoundError(f"'{JSONL_PATH}' bulunamadı. Önceki hücreyi çalıştırın.")

try:
    # Dosyayı OpenAI'ye yükle
    with open(JSONL_PATH, "rb") as f:
        uploaded_file = client.files.create(file=f, purpose="fine-tune")
    file_id = uploaded_file.id
    print(f"Yüklendi → File ID: {file_id}")

    # Fine-tuning başlat
    job = client.fine_tuning.jobs.create(
        training_file=file_id,
        model="gpt-4o-mini-2024-07-18"
    )
    job_id = job.id
    print(f"Başlatıldı → Job ID: {job_id}")
    print(f"Takip: https://platform.openai.com/fine-tuning/{job_id}")

    # Global değişken olarak kaydet
    global FINE_TUNING_JOB_ID
    FINE_TUNING_JOB_ID = job_id

except openai.APIError as e:
    # Kota/bakiye hatalarını yakala
    if "exceeded_quota" in str(e).lower() or "insufficient_quota" in str(e).lower():
        print("UYARI: Bakiye yetersiz veya kota aşıldı.")
        print("Lütfen OpenAI hesabınızı kontrol edin.")
    else:
        print(f"API Hatası: {e}")
except Exception as e:
    # Diğer hataları yakala
    print(f"Beklenmeyen Hata: {e}")


Yüklendi → File ID: file-P9qwRgL1fsyqy5yiCFvihj
UYARI: Bakiye yetersiz veya kota aşıldı.
Lütfen OpenAI hesabınızı kontrol edin.


In [237]:
# 06_cost_estimation | Toplam örnek ve token sayısını hesapla

import sys, subprocess, json, os
from pathlib import Path

# tiktoken yüklü değilse kur
try:
    import tiktoken
except ImportError:
    print("'tiktoken' yükleniyor...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "tiktoken"])
    import tiktoken

# Tokenizer ayarla (kullandığın modelle uyumlu)
encoding = tiktoken.encoding_for_model("gpt-4o-mini-2024-07-18")

# JSONL dosyası tanımlı olmalı
if not os.path.exists(JSONL_PATH):
    raise FileNotFoundError(f"'{JSONL_PATH}' bulunamadı. Önceki hücreyi çalıştır.")

jsonl_file_path = Path(JSONL_PATH)

# Sayım değişkenleri
total_tokens = 0
total_examples = 0

# Satır satır oku, her mesajın toplam token'ını hesapla
with open(jsonl_file_path, "r", encoding="utf-8") as f:
    for line in f:
        data = json.loads(line)
        messages = data["messages"]
        msg_text = "".join(m["content"] for m in messages)
        tokens = len(encoding.encode(msg_text))
        total_tokens += tokens
        total_examples += 1

# Çıktıyı yazdır
print(f"Toplam örnek sayısı: {total_examples}")
print(f"Toplam token sayısı: {total_tokens}")


Toplam örnek sayısı: 2531
Toplam token sayısı: 2215587


In [238]:
# 07_follow_job | Fine-tuning iş durumunu izle ve model adını kaydet

import time
from datetime import datetime

# FINE_TUNING_JOB_ID önceki hücrede (05) tanımlanır.
# Tanımlı değilse hata ver
if 'FINE_TUNING_JOB_ID' not in globals():
    raise NameError("FINE_TUNING_JOB_ID tanımlı değil. Lütfen 05. hücreyi çalıştırın.")

# İş durumu tamamlanana veya başarısız olana kadar izle
while True:
    job_info = client.fine_tuning.jobs.retrieve(FINE_TUNING_JOB_ID)
    status = job_info.status
    print(datetime.now().strftime("%H:%M:%S"), "→", status)
    
    if status in ("succeeded", "failed"):
        break
    time.sleep(15) # 15 saniye bekle

if status == "succeeded":
    model_name = job_info.fine_tuned_model
    print("Model hazır →", model_name)

    # Model adını dosyaya kaydet
    with open("fine_tuned_model.txt", "w") as f:
        f.write(model_name)
    print("Model adı 'fine_tuned_model.txt' dosyasına yazıldı.")
else:
    print(f"Eğitim başarısız: {FINE_TUNING_JOB_ID}")

NameError: FINE_TUNING_JOB_ID tanımlı değil. Lütfen 05. hücreyi çalıştırın.

In [239]:
# 08_model_test | Fine-tuned modele örnek prompt gönder ve yanıtı al

import openai, json

# Sistem mesajı (GPT'nin nasıl davranması gerektiği tanımlanır)
SYSTEM_PROMPT = (
    "You are an AI assistant representing the internal personnel of a service platform. "
    "Your role is to review refund requests submitted by service providers and make the final decision: "
    "'Accepted by Personnel' or 'Rejected by Personnel'. "
    "You must base your decision on the following structured information:\n\n"
    "- Reason for Refund Request: A short written justification from the provider.\n"
    "- Service Provider Call Transcripts: Transcript(s) of provider's calls with the customer.\n"
    "- Service Provider Call Count: How many times the provider attempted to reach the customer (based on call segments).\n"
    "- Partner Call Transcripts: Transcript(s) of the partner's confirmation calls with the customer.\n"
    "- Partner Call Count: How many times the partner called the customer for confirmation (based on call segments).\n"
    "- Time Elapsed Between Job Submission and Refund Request: Time in minutes.\n\n"
    "Analyze whether the provider gives a valid, timely, and reasonable explanation. "
    "Reject vague, insufficient, abusive claims, or cases where fault lies with the provider. "
    "Accept cases with clear external causes (e.g., customer unavailable, location inaccessible, technical issues beyond provider control). "
    "Do not assume anything beyond the given data.\n\n"
    "Return explanation in Turkish. Cevap açıklaması kısa ve Türkçe olmalıdır.\n\n"
    "Respond ONLY with a JSON object:\n"
    "{'decision': 'Accepted by Personnel' or 'Rejected by Personnel', 'reason': 'short explanation in Turkish'}"
)

# Fine-tuned model adını dosyadan al
try:
    with open("fine_tuned_model.txt", "r") as f:
        final_model_name = f.read().strip()
    print("Kullanılacak model:", final_model_name)
except FileNotFoundError:
    print("UYARI: 'fine_tuned_model.txt' bulunamadı.")
    final_model_name = None

# Modele test promptu gönderip yanıt döndüren fonksiyon
def evaluate(model_name: str,
            reason: str,
            provider_transcripts: str = "",
            partner_transcripts: str = "",
            provider_call_count: int = 0,
            partner_call_count: int = 0,
            elapsed_minutes: int = 0) -> dict:
    
    if not model_name:
        raise ValueError("Model adı tanımlı değil.")

    user_prompt = (
        f"Reason for Refund Request: {reason}\n"
        f"Service Provider Call Transcripts: {provider_transcripts}\n"
        f"Service Provider Call Count: {provider_call_count}\n"
        f"Partner Call Transcripts: {partner_transcripts}\n"
        f"Partner Call Count: {partner_call_count}\n"
        f"Time Elapsed Between Job Submission and Refund Request: {elapsed_minutes} minutes."
    )

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)

    except json.JSONDecodeError as e:
        print(f"Geçersiz JSON: {e}")
        print("Yanıt:", response.choices[0].message.content)
        return {"decision": "ERROR", "reason": "Geçersiz JSON"}

    except openai.APIError as e:
        if "quota" in str(e).lower():
            print("Quota hatası: Bakiye yetersiz.")
        else:
            print(f"OpenAI API hatası: {e}")
        return {"decision": "API_ERROR", "reason": str(e)}

    except Exception as e:
        print(f"Genel hata: {e}")
        return {"decision": "UNKNOWN_ERROR", "reason": str(e)}

# Örnek test verisiyle çağrı
test_reason = "Müşteri yanlış adres vermiş, hizmet iptal edilmiş."
test_provider_transcripts = "Müşteri adresi Zeytinburnu demişti ama aradığım kişi Erzurum'daydı."
test_partner_transcripts = "Partner: Müşteri adres bilgisinin yanlış olduğunu doğruladı."

result = evaluate(
    model_name=final_model_name,
    reason=test_reason,
    provider_transcripts=test_provider_transcripts,
    partner_transcripts=test_partner_transcripts,
    provider_call_count=2,
    partner_call_count=1,
    elapsed_minutes=22
)

# Sonucu yazdır
print("\n--- Model Kararı ---")
print("Karar   :", result.get("decision", "Belirsiz"))
print("Gerekçe :", result.get("reason", "Belirsiz"))


UYARI: 'fine_tuned_model.txt' bulunamadı.


ValueError: Model adı tanımlı değil.

In [240]:
# 09_eval_set_analysis | Modeli küçük bir eval set ile test et

import random
import pandas as pd

# Eval için 100 rastgele kayıt seç
eval_sample = df.sample(n=100, random_state=42).copy()

eval_results = []
for _, row in eval_sample.iterrows():
    expected_decision = (
        "Accepted by Personnel"
        if row["İade Durumu"] == "Personel Kabul Etti"
        else "Rejected by Personnel"
    )

    # Model tahmini al
    response = evaluate(
        final_model_name,
        reason=row["İade Talebi Nedeni"],
        provider_transcripts=row["Hizmet Veren Arama Metinleri"],
        partner_transcripts=row["Partner Arama Metinleri"],
        provider_call_count=row["HizmetVerenAramaSayisi"],
        partner_call_count=row["PartnerAramaSayisi"],
        elapsed_minutes=0 if pd.isna(row["Time_Elapsed_Minutes"]) else int(row["Time_Elapsed_Minutes"])
    )

    predicted_decision = response.get("decision", "").strip()
    reason_text = response.get("reason", "").strip()

    eval_results.append({
        "expected_decision": expected_decision,
        "predicted_decision": predicted_decision,
        "reason": reason_text,
        "correct": expected_decision.lower() == predicted_decision.lower()
    })

# Doğruluk oranı hesapla
correct_count = sum(1 for r in eval_results if r["correct"])
total_eval_samples = len(eval_results)
accuracy = correct_count / total_eval_samples
print(f"\nEval Set Doğruluğu: {accuracy:.2%} ({correct_count}/{total_eval_samples})")

# Hatalı tahminleri yazdır
print("\nHatalı Tahminler:")
for r in eval_results:
    if not r["correct"]:
        print(f"Beklenen: {r['expected_decision']} | Tahmin Edilen: {r['predicted_decision']}")
        print(f"Gerekçe: {r['reason']}")
        print("-" * 60)


ValueError: Model adı tanımlı değil.

In [241]:
# 10_eval_metrics | Sınıflandırma metriklerini hesapla

# scikit-learn varsa kullan, yoksa yükle
try:
    from sklearn.metrics import classification_report
except ImportError:
    import subprocess, sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-learn"])
    from sklearn.metrics import classification_report

# Etiketleri normalize et (lowercase, trim)
expected = [r["expected_decision"].strip().lower() for r in eval_results]
predicted = [r["predicted_decision"].strip().lower() for r in eval_results]

# Liste boşsa uyar
if not expected or not predicted:
    print("Uyarı: Değerlendirme listeleri boş.")
else:
    # Metrikleri yazdır
    print("Sınıflandırma Raporu:\n")
    print(classification_report(expected, predicted, digits=3, labels=["accepted by personnel", "rejected by personnel"]))


Uyarı: Değerlendirme listeleri boş.


In [242]:
# 11_error_analysis | Hatalı tahminlerde model gerekçesini detaylı analiz et

print("Hatalı Tahminler - Detaylı İnceleme:\n")

for i, row in enumerate(eval_results):
    if not row["correct"]:
        print(f"#{i+1}")
        print("Beklenen Karar :", row["expected_decision"])
        print("Model Tahmini  :", row["predicted_decision"])
        print("Model Gerekçesi:", row.get("reason", "(Açıklama bulunamadı)"))
        print("-" * 70)


Hatalı Tahminler - Detaylı İnceleme:



In [243]:
# 12_export_eval | Eval sonuçlarını CSV dosyasına dışa aktar

import pandas as pd

# DataFrame oluştur
eval_df = pd.DataFrame(eval_results)

# Sütun sırasını elle belirle (görsel temizlik)
columns = [
    "expected_decision",
    "predicted_decision",
    "correct",
    "reason"
]
# Sadece bu sütunları al, varsa diğerlerini de bırak
eval_df = eval_df[[col for col in columns if col in eval_df.columns]]

# CSV dosyasına yaz
eval_df.to_csv("eval_results.csv", index=False, encoding="utf-8-sig")

print("Eval sonuçları CSV dosyasına yazıldı → eval_results.csv")


Eval sonuçları CSV dosyasına yazıldı → eval_results.csv
